In [ ]:
import lmstudio as lms
import json
import time
import pandas as pd
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME = "qwen3-8b-awq"

In [ ]:
def verificar_qwen3_local():
    """Verifica que LM Studio esté corriendo y disponible"""
    try:
        print("🔍 Verificando conexión con LM Studio...")
        # Conectar usando el cliente por defecto de LM Studio
        model = lms.llm()
        print("✅ Conexión a LM Studio exitosa")
        return model
    except Exception as e:
        print(f"❌ Error conectando a LM Studio: {e}")
        print("💡 PASOS PARA SOLUCIONAR:")
        print("   1. Abrir LM Studio desktop")
        print("   2. Ir a pestaña 'Local Server'")
        print("   3. Cargar cualquier modelo Qwen (ej: Qwen2.5-7B-Instruct)")
        print("   4. Hacer clic en 'Start Server'")
        print("   5. Verificar que dice 'Server running on port 1234'")
        print("   6. Ejecutar este código nuevamente")
        return None

def obtener_info_modelo(model):
    """Obtiene información del modelo local"""
    try:
        test_response = model.respond("Hello, respond with 'Model working'")
        print("📋 Información del modelo:")
        print(f"   ✅ Estado: Funcionando correctamente")
        print(f"   🧪 Test: {test_response}")
        return True
    except Exception as e:
        print(f"❌ Error en test del modelo: {e}")
        return False

# Verificación inicial
print("🔍 Verificando LM Studio local...")
model = verificar_qwen3_local()

if model:
    modelo_ok = obtener_info_modelo(model)
else:
    modelo_ok = False
    print("\\n🛠️ INSTRUCCIONES COMPLETAS:")
    print("   1. Descargar e instalar LM Studio desde https://lmstudio.ai/")
    print("   2. Abrir LM Studio")
    print("   3. En 'Discover', buscar y descargar un modelo Qwen")
    print("   4. En 'Local Server', cargar el modelo y hacer 'Start Server'")
    print("   5. Ejecutar este notebook nuevamente")

In [ ]:
def resumir_documento_filatelico(texto, tipo_resumen="completo", max_tokens=800):
    """Resume documentos filatélicos con Qwen3-8B-AWQ local"""
    
    if not model:
        print("❌ Modelo no disponible")
        return None
    
    # Prompts según tipo de resumen
    if tipo_resumen == "completo":
        prompt = f"Analyze this philatelic document and create a structured summary. Include HISTORICAL, TECHNICAL, ISSUE, PHILATELIC and ERRORS sections only if data is present. Document: {texto}"
    elif tipo_resumen == "tecnico":
        prompt = f"Como especialista técnico filatélico, analiza métodos de impresión, dentados, variantes de color y conservación. Documento: {texto}"
    elif tipo_resumen == "historico":
        prompt = f"Como historiador filatélico costarricense, analiza contexto histórico, eventos conmemorados y figuras históricas. Documento: {texto}"
    else:
        prompt = f"Resume este documento filatélico en máximo 3 párrafos con lo más importante: {texto}"
    
    system_prompt = "Eres un experto filatélico especializado en Costa Rica con amplio conocimiento en historia, técnicas de impresión y valoración de estampillas."
    full_prompt = f"{system_prompt} {prompt} /no_think"
    
    try:
        print(f"🤖 Generando resumen {tipo_resumen} con LM Studio...")
        start_time = time.time()
        
        # Llamar al modelo y obtener el resultado
        result = model.respond(full_prompt)
        
        # Extraer el texto de la respuesta del PredictionResult
        if hasattr(result, 'text'):
            resumen = result.text
        elif hasattr(result, 'content'):
            resumen = result.content
        else:
            # Si no tiene los atributos esperados, convertir a string
            resumen = str(result)
        
        generation_time = time.time() - start_time
        estimated_tokens = len(resumen.split()) * 1.3
        
        print(f"✅ Resumen generado en {generation_time:.2f} segundos")
        print(f"📊 Tokens estimados: ~{int(estimated_tokens)}")
        
        if generation_time > 0:
            velocidad = estimated_tokens / generation_time
            print(f"🚀 Velocidad estimada: ~{velocidad:.1f} tokens/segundo")
        
        return {
            'tipo': tipo_resumen,
            'resumen': resumen,
            'tiempo': generation_time,
            'tokens_estimados': int(estimated_tokens),
            'velocidad': velocidad if 'velocidad' in locals() else 0
        }
        
    except Exception as e:
        print(f"❌ Error generando resumen: {e}")
        print(f"🔍 Tipo de resultado: {type(result) if 'result' in locals() else 'No definido'}")
        print(f"🔍 Atributos disponibles: {dir(result) if 'result' in locals() else 'No disponible'}")
        return None

print("✅ Función de resumen filatélico con LM Studio actualizada")

In [ ]:
# Textos de ejemplo sobre filatelia costarricense
textos_filatelicos = {
    "ejemplo_1": "PARTE SEGUNDA - LA PLANCHA. La plancha usada para cancelar con las barras los estampillas enumeradas se componía de 5 grupos de barras paralelas de 5 líneas cada uno. Es posible que el método de impresión fuera el de imprimir la mitad de un pliego completo de 100 estampillas y posteriormente se dio vuelta al pliego para imprimir la mitad restante. Una explicación más lógica se apoya en la teoría sustentada por el ing. Peralta: los pliegos doblados fueron insertados en la prensa y luego se voltearon.",
    
    "ejemplo_2": "From Hind to Woo Inverted Center 2c stamp of 1901 by Fred O'Neill. The inverted center stamp 2c of 1901 is the only known copy. This singular stamp is mint with perforation 15 of the same normal color of the regular issued which was engraved and printed in sheets of 100 by Waterlow & Sons of London. The stamp was sold for 12 pounds sterling in 1935 and later sold to Norman Hubbard for $550.00 in 1961.",
    
    "ejemplo_3": "The Costa Rica 1881-82 and the 1883 UPU Surcharge Forgeries/Fantasies (2015). It is a 52 page monograph by Raul Hernandez showing in detail the material described in the title. The booklet is in color in high quality laser paper. Price $20.00 plus postage, order it from the author. First Edition 2015."
}

print("📚 Textos filatélicos de prueba cargados:")
for clave, texto in textos_filatelicos.items():
    print(f"   📄 {clave}: {len(texto)} caracteres")

In [ ]:
resultados_resumenes = []

if modelo_ok:
    print("🔬 EJECUTANDO TESTS DE RESUMEN FILATÉLICO CON LM STUDIO LOCAL")
    print("="*60)
    
    # Test 1: Resumen completo del texto histórico
    print("\n📖 TEST 1: RESUMEN HISTÓRICO COMPLETO")
    print("-" * 40)
    resultado1 = resumir_documento_filatelico(
        textos_filatelicos["ejemplo_1"], 
        tipo_resumen="completo",
        max_tokens=500
    )
    
    if resultado1:
        resultados_resumenes.append(resultado1)
        print(f"\n📝 RESUMEN GENERADO:")
        print(resultado1['resumen'])
    
    # Test 2: Resumen de estampilla invertida
    print("\n" + "="*60)
    print("\n📖 TEST 2: RESUMEN DE ESTAMPILLA INVERTIDA")
    print("-" * 40)
    resultado2 = resumir_documento_filatelico(
        textos_filatelicos["ejemplo_2"], 
        tipo_resumen="completo",
        max_tokens=500
    )
    
    if resultado2:
        resultados_resumenes.append(resultado2)
        print(f"\n📝 RESUMEN GENERADO:")
        print(resultado2['resumen'])
    
    # Test 3: Resumen de monografía
    print("\n" + "="*60)
    print("\n📖 TEST 3: RESUMEN DE MONOGRAFÍA")
    print("-" * 40)
    resultado3 = resumir_documento_filatelico(
        textos_filatelicos["ejemplo_3"], 
        tipo_resumen="completo",
        max_tokens=500
    )
    
    if resultado3:
        resultados_resumenes.append(resultado3)
        print(f"\n📝 RESUMEN GENERADO:")
        print(resultado3['resumen'])
    
    # Mostrar estadísticas finales
    if resultados_resumenes:
        print("\n" + "="*60)
        print("📊 ESTADÍSTICAS DE RENDIMIENTO")
        print("-" * 40)
        
        tiempo_total = sum([r['tiempo'] for r in resultados_resumenes])
        tokens_total = sum([r['tokens_estimados'] for r in resultados_resumenes])
        velocidad_promedio = sum([r['velocidad'] for r in resultados_resumenes]) / len(resultados_resumenes)
        
        print(f"⏱️ Tiempo total: {tiempo_total:.2f} segundos")
        print(f"📝 Tokens totales: ~{tokens_total}")
        print(f"🚀 Velocidad promedio: ~{velocidad_promedio:.1f} tokens/segundo")
        print(f"✅ Tests completados: {len(resultados_resumenes)}/3")

else:
    print("⚠️ Saltando tests - Qwen3-8B-AWQ no disponible")
    print("💡 Configura LM Studio y descarga el modelo para continuar")